In [ ]:
!pip install ultralytics opencv-python numpy tqdm

In [ ]:
import os
import shutil
import numpy as np
from tqdm import tqdm
import time
import cv2
from ultralytics import YOLO
from ultralytics import YOLO


In [ ]:

kitti_base_path = "/kaggle/input/kitti-dataset"

kitti_images_dir = os.path.join(kitti_base_path, "data_object_image_2/training/image_2")
kitti_labels_dir = os.path.join(kitti_base_path, "data_object_label_2/training/label_2")
yolo_dataset_path = "yolo_dataset"
yolo_images_train = os.path.join(yolo_dataset_path, "images/train")
yolo_images_val = os.path.join(yolo_dataset_path, "images/val")
yolo_labels_train = os.path.join(yolo_dataset_path, "labels/train")
yolo_labels_val = os.path.join(yolo_dataset_path, "labels/val")
os.makedirs(yolo_images_train, exist_ok=True)
os.makedirs(yolo_images_val, exist_ok=True)
os.makedirs(yolo_labels_train, exist_ok=True)
os.makedirs(yolo_labels_val, exist_ok=True)

print("Directories set up correctly.")


In [ ]:

kitti_classes = ["Car", "Pedestrian", "Cyclist", "Truck", "Van","Misc"]
def convert_kitti_to_yolo(label_file, split="train"):
    with open(os.path.join(kitti_labels_dir, label_file), "r") as f:
        lines = f.readlines()
    
    new_lines = []
    for line in lines:
        parts = line.strip().split()
        obj_class = parts[0]
        if obj_class in kitti_classes:
            x_min, y_min, x_max, y_max = map(float, parts[4:8])
            width = 1242 
            height = 375

            x_center = ((x_min + x_max) / 2) / width
            y_center = ((y_min + y_max) / 2) / height
            bbox_width = (x_max - x_min) / width
            bbox_height = (y_max - y_min) / height

            yolo_class = kitti_classes.index(obj_class)
            new_lines.append(f"{yolo_class} {x_center} {y_center} {bbox_width} {bbox_height}\n")

    yolo_label_path = yolo_labels_train if split == "train" else yolo_labels_val
    with open(os.path.join(yolo_label_path, label_file), "w") as f:
        f.writelines(new_lines)

image_files = sorted(os.listdir(kitti_images_dir))
split_idx = int(0.8 * len(image_files))

train_files = image_files[:split_idx]
val_files = image_files[split_idx:]

for file in tqdm(train_files, desc="Processing Training Data"):
    convert_kitti_to_yolo(file.replace('.png', '.txt'), "train")
    shutil.copy(os.path.join(kitti_images_dir, file), os.path.join(yolo_images_train, file))

for file in tqdm(val_files, desc="Processing Validation Data"):
    convert_kitti_to_yolo(file.replace('.png', '.txt'), "val")
    shutil.copy(os.path.join(kitti_images_dir, file), os.path.join(yolo_images_val, file))

print("KITTI annotations converted and dataset split.")


In [ ]:
dataset_yaml = f"""\
path: /kaggle/working/yolo_dataset
train: images/train
val: images/val
names:
  0: Car
  1: Pedestrian
  2: Cyclist
  3: Truck
  4: Van
  5: Misc
"""

with open("/kaggle/working/dataset.yaml", "w") as f:
    f.write(dataset_yaml)

print("Updated dataset.yaml file created")

In [ ]:
model = YOLO("yolov8n.pt")  
model.train(data="dataset.yaml", epochs=25, imgsz=640)

In [ ]:
test_image = "/kaggle/input/kitti-dataset/data_object_image_2/testing/image_2/000217.png" 

results = model(test_image)
results[0].show()

In [ ]:
metrics = model.val()
mAP_50 = metrics.box.map50   
mAP_50_95 = metrics.box.map  

print(f"Model Accuracy (mAP@50): {mAP_50:.2f}")
print(f"Model Accuracy (mAP@50-95): {mAP_50_95:.2f}")

In [ ]:

model = YOLO("/kaggle/input/yolobest/pytorch/default/1/best.pt", task="detect")
results = model.predict(source="/kaggle/input/kitti-dataset/data_object_image_2/testing/image_2/000217.png", imgsz=640)
res = results[0]
res.show()


In [ ]:
model = YOLO("/kaggle/input/yolobest/pytorch/default/1/best.pt", task="detect")
img = cv2.imread("/kaggle/input/kitti-dataset/data_object_image_2/testing/image_2/000217.png")
for _ in range(5):
    _ = model.predict(source=img, imgsz=640, device=0)
N = 50 
start = time.time()
for _ in range(N):
    _ = model.predict(source=img, imgsz=640, device=0)
end = time.time()
total_time = end - start
fps = N / total_time
print(f"Average inference time per image: {total_time/N*1000:.1f} ms")
print(f"=> Approximate FPS: {fps:.1f}")
